In [1]:
%matplotlib inline

import pandas as pd
import os
import re
from dateutil.parser import parse
from datetime import datetime
import random
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('ggplot')

In [2]:
def FindNumbers(input_string):
    
    try:
        numbers_list = re.findall(r'\d+', input_string)
    except:
        numbers_list = []
        
    return numbers_list

In [3]:
def GetFirst(input_list):
    if len(input_list) > 0:
        first = input_list[0]
    else:
        first = ''
    return first

In [4]:
def ConvertDate(input_string):
    
    try:
        conv_date = parse(input_string)
    except:
        conv_date = input_string
    return conv_date

In [5]:
def DateConv(date_str):
    '''
    Input: 'Y-m-d H:M:S'
    '''
    
    ## datetime object
    date_object = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

    ## timestamp
    ts = datetime.timestamp(date_object) 
    
    
    return ts, date_object

In [6]:
def DateCorrector(test_date, desired_month):

    ## the month we know it is correct: desired_month

    ## the month we need to check:
    test_month = str(test_date.month)

    if test_month != desired_month:
        ## inverting day and month
        true_month = str(test_date.day)
        true_day = str(test_date.month)
    else:
        ## keeping month and day
        true_month = test_month
        true_day = str(test_date.day)

    date = '-'.join([str(test_date.year), str(true_month), str(true_day)]) + ' 0:0:0'
    


    corrected_date = DateConv(date)[1]
    
    
    return corrected_date


In [7]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [8]:
cwd = os.getcwd()
print(cwd)

/home/dev/sigo_prod


In [9]:
path_data = os.path.join(cwd, '_data_original')
path_output = os.path.join(cwd, '_data_output')

In [10]:
os.listdir(path_data)

['main_trix_v2.xlsx',
 'oficios_google.xlsx',
 'main_trix.xlsx',
 'pastas_linhas',
 'emails.csv',
 'controle_linhas.xlsx']

### 'Oficios Google':

In [11]:
oficios_xlsx = pd.ExcelFile(os.path.join(path_data, 'oficios_google.xlsx'))

In [12]:
OficiosSheets_list = oficios_xlsx.sheet_names
print(OficiosSheets_list)

['dez2018', 'jan2019', 'Fev2019', 'Mar2019', 'Junho2019', 'Apr2019', 'Maio2019', 'Julho2019', 'Ago2019', 'Set2019', 'Out2019', 'Nov2019', 'Dez2019']


In [13]:
month_dict = {'dez2018': '12',
             'jan2019': '1',
             'Fev2019': '2',
             'Mar2019': '3',
             'Apr2019': '4',
             'Maio2019': '5',
             'Junho2019': '6',
             'Julho2019': '7',
             'Ago2019': '8',
             'Set2019': '9',
             'Out2019': '10',
             'Nov2019': '11',
             'Dez2019':'12'}

In [14]:
## reading all "oficios google" xlsx sheets:
df_oficios_list = []



counter = 0
for sheet in OficiosSheets_list:
    
    
    if counter == 0:
        df_temp = pd.read_excel(os.path.join(path_data, 'oficios_google.xlsx'),\
                                sheet_name = sheet,
                                parse_dates=True, dayfirst = True)

        columns_list = df_temp.columns
        columns_list = [item.replace('\n', ' ') for item in columns_list]
    
    else:
        df_temp = pd.read_excel(os.path.join(path_data, 'oficios_google.xlsx'),\
                                sheet_name = sheet,
                                parse_dates=True, dayfirst = True, header = None)   
        
        
    if len(df_temp.columns) == 7:
        df_temp = df_temp.drop(df_temp.columns[6], axis = 1)
        
    
    
    if type(df_temp.index[0]) != int:
        df_temp = df_temp.reset_index()
        
        
    df_temp.columns = columns_list
    
    
    
    ## Month correction step:
    desired_month = month_dict[sheet]
    print(desired_month)
    date_column = 'Recebimento'
    #df_temp = pd.read_excel(os.path.join(path_data, 'oficios_google.xlsx'),\
    #                        sheet_name = sheet, dtype = {date_column: str})
    ## dropping nan
    df_temp = df_temp.dropna(subset = [date_column]).reset_index(drop = True)
    ## checking if there is numbers in the date field:
    
    try:
        df_temp = df_temp[df_temp[date_column].apply(hasNumbers) == True].reset_index(drop = True)

    except:
        pass
    
    
    df_temp = df_temp[df_temp[date_column] != ' '].reset_index(drop = True)
    df_temp = df_temp[df_temp[date_column] != '  '].reset_index(drop = True)
    df_temp = df_temp[df_temp[date_column] != '#'].reset_index(drop = True)
    df_temp = df_temp[df_temp[date_column].apply(type) != int].reset_index(drop = True)
    
    ## converting to datetime:
    df_temp[date_column] = pd.to_datetime(df_temp[date_column])


    ## correcting the month: 
    df_temp[date_column] = df_temp[date_column].apply(DateCorrector, desired_month = desired_month)
    
    
    
    
    df_oficios_list.append(df_temp)
    counter += 1

12
1
2
3
6
4
5
7
8
9
10
11
12


In [15]:
df_oficios_list[5]

,Recebimento,Assunto,Ofício,Advogado,Info,Status
0,2019-04-01,FWD: [4-7856000026263] BR Data Request 0002268...,00002268-16.2018.8.26.0125,Rafael,relacionado #4976,#5169
1,2019-04-01,FWD: [5-6699000025849] Decisão Judicial - 4ª V...,Oficio PR_PA_GAB 5_N 1793_2019\nOficio_GABJU_4...,Rafael,caso novo,#5170
2,2019-04-01,FWD: [6-5304000026212] BR Data Request 1519239...,1519239-91.2018.8.26.0451,Rafael,caso novo,#5173
3,2019-04-01,FWD: [8-6982000025791] BR Data Request 010/2.1...,1014_2019,Rafael,caso novo,#5174
4,2019-04-01,FWD: [8-5041000025342] BR Data Request 0001948...,0001948-35.2019.8.16.0019\n573_2019,Rafael,relacionado #4905,respondido ao LIS
...,...,...,...,...,...,...
445,2019-04-30,FWD: [5-8091000026135] Solicitação de dados ca...,NaN,Helena,relacionado # 5180,#5425
446,2019-04-30,RE: [4-7597000026554] URGENTE- DECISÃO/OFÍCIO ...,NaN,Helena,relacionado #5412,duplicidade
447,2019-04-30,FWD: [4-5810000024453] Ofício nº 359/2019 - Cu...,NaN,Helena,relacionado #4541,respondido ao LIS
448,2019-04-30,FWD: [9-7751000026138] Resposta ao seu pedido ...,NaN,Rafael,relacionado #5334,respondido ao LIS


In [16]:
df_oficios = pd.concat(df_oficios_list).dropna(how = 'all').reset_index(drop = True)

In [17]:
df_oficios

,Recebimento,Assunto,Ofício,Advogado,Info,Status
0,2018-12-04,FWD: [6-5840000024315] Quebra de Sigilo - URGE...,NaN,Luisa,Caso novo,Enviado #4512
1,2018-12-04,RE_ _1-9575000024540_ BR Data Request 00009613...,NaN,Rafael,Extensão #4193,Enviado advice #4515
2,2018-12-04,FWD: [6-4781000024836] Afastamento do sigilo t...,NaN,Luisa,Caso novo,Enviado #4513
3,2018-12-04,RES: [3-2530000024386] BR Data Request 0004435...,NaN,Rafael,Extensão #4358,Enviado advice #4516
4,2018-12-04,Fwd: [juridicobrasil] Quebra de sigilo telemát...,NaN,Luisa,Email indicando autoridade #4481,enviado #4481
...,...,...,...,...,...,...
5898,2019-12-27,FWD: [7-2066000028626] [BRA] - [Official Lette...,006-2019,Helena,relacionado #7739,respondido ao LIS
5899,2019-12-26,Re: [6-6324000028920] - c 2019/12/27 - Decembe...,NaN,Helena,relacionado #7771,respondido ao LIS
5900,2019-12-26,Re: [3-0569000028501] PETITION - c 2020/01/06 ...,NaN,Helena,relacionado #7774,respondido ao LIS
5901,2019-12-27,FWD: [9-9053000029127] BR Data Request: 0245.1...,627-2019\nOficio Judicial - 0245.19.007487-3,Helena,caso novo,#7784


In [18]:
df_oficios['line'] = df_oficios['Status'].str.replace('#', '').apply(FindNumbers).apply(GetFirst)

In [19]:
df_oficios

,Recebimento,Assunto,Ofício,Advogado,Info,Status,line
0,2018-12-04,FWD: [6-5840000024315] Quebra de Sigilo - URGE...,NaN,Luisa,Caso novo,Enviado #4512,4512
1,2018-12-04,RE_ _1-9575000024540_ BR Data Request 00009613...,NaN,Rafael,Extensão #4193,Enviado advice #4515,4515
2,2018-12-04,FWD: [6-4781000024836] Afastamento do sigilo t...,NaN,Luisa,Caso novo,Enviado #4513,4513
3,2018-12-04,RES: [3-2530000024386] BR Data Request 0004435...,NaN,Rafael,Extensão #4358,Enviado advice #4516,4516
4,2018-12-04,Fwd: [juridicobrasil] Quebra de sigilo telemát...,NaN,Luisa,Email indicando autoridade #4481,enviado #4481,4481
...,...,...,...,...,...,...,...
5898,2019-12-27,FWD: [7-2066000028626] [BRA] - [Official Lette...,006-2019,Helena,relacionado #7739,respondido ao LIS,
5899,2019-12-26,Re: [6-6324000028920] - c 2019/12/27 - Decembe...,NaN,Helena,relacionado #7771,respondido ao LIS,
5900,2019-12-26,Re: [3-0569000028501] PETITION - c 2020/01/06 ...,NaN,Helena,relacionado #7774,respondido ao LIS,
5901,2019-12-27,FWD: [9-9053000029127] BR Data Request: 0245.1...,627-2019\nOficio Judicial - 0245.19.007487-3,Helena,caso novo,#7784,7784


In [20]:
df_oficios['Recebimento'] = df_oficios['Recebimento'].apply(ConvertDate)

In [21]:
df_oficios

,Recebimento,Assunto,Ofício,Advogado,Info,Status,line
0,2018-12-04,FWD: [6-5840000024315] Quebra de Sigilo - URGE...,NaN,Luisa,Caso novo,Enviado #4512,4512
1,2018-12-04,RE_ _1-9575000024540_ BR Data Request 00009613...,NaN,Rafael,Extensão #4193,Enviado advice #4515,4515
2,2018-12-04,FWD: [6-4781000024836] Afastamento do sigilo t...,NaN,Luisa,Caso novo,Enviado #4513,4513
3,2018-12-04,RES: [3-2530000024386] BR Data Request 0004435...,NaN,Rafael,Extensão #4358,Enviado advice #4516,4516
4,2018-12-04,Fwd: [juridicobrasil] Quebra de sigilo telemát...,NaN,Luisa,Email indicando autoridade #4481,enviado #4481,4481
...,...,...,...,...,...,...,...
5898,2019-12-27,FWD: [7-2066000028626] [BRA] - [Official Lette...,006-2019,Helena,relacionado #7739,respondido ao LIS,
5899,2019-12-26,Re: [6-6324000028920] - c 2019/12/27 - Decembe...,NaN,Helena,relacionado #7771,respondido ao LIS,
5900,2019-12-26,Re: [3-0569000028501] PETITION - c 2020/01/06 ...,NaN,Helena,relacionado #7774,respondido ao LIS,
5901,2019-12-27,FWD: [9-9053000029127] BR Data Request: 0245.1...,627-2019\nOficio Judicial - 0245.19.007487-3,Helena,caso novo,#7784,7784


In [22]:
idx2drop_list = df_oficios[df_oficios['Assunto'] == 'Assunto'].index
print(idx2drop_list)
df_oficios = df_oficios.drop(idx2drop_list, axis = 0).reset_index(drop = True)


idx2drop_list = df_oficios[df_oficios['Recebimento'] == '#'].index
print(idx2drop_list)
df_oficios = df_oficios.drop(idx2drop_list, axis = 0).reset_index(drop = True)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


In [23]:
df_oficios

,Recebimento,Assunto,Ofício,Advogado,Info,Status,line
0,2018-12-04,FWD: [6-5840000024315] Quebra de Sigilo - URGE...,NaN,Luisa,Caso novo,Enviado #4512,4512
1,2018-12-04,RE_ _1-9575000024540_ BR Data Request 00009613...,NaN,Rafael,Extensão #4193,Enviado advice #4515,4515
2,2018-12-04,FWD: [6-4781000024836] Afastamento do sigilo t...,NaN,Luisa,Caso novo,Enviado #4513,4513
3,2018-12-04,RES: [3-2530000024386] BR Data Request 0004435...,NaN,Rafael,Extensão #4358,Enviado advice #4516,4516
4,2018-12-04,Fwd: [juridicobrasil] Quebra de sigilo telemát...,NaN,Luisa,Email indicando autoridade #4481,enviado #4481,4481
...,...,...,...,...,...,...,...
5898,2019-12-27,FWD: [7-2066000028626] [BRA] - [Official Lette...,006-2019,Helena,relacionado #7739,respondido ao LIS,
5899,2019-12-26,Re: [6-6324000028920] - c 2019/12/27 - Decembe...,NaN,Helena,relacionado #7771,respondido ao LIS,
5900,2019-12-26,Re: [3-0569000028501] PETITION - c 2020/01/06 ...,NaN,Helena,relacionado #7774,respondido ao LIS,
5901,2019-12-27,FWD: [9-9053000029127] BR Data Request: 0245.1...,627-2019\nOficio Judicial - 0245.19.007487-3,Helena,caso novo,#7784,7784


In [24]:
idx2drop_list2 = []
for k in range(0, len(df_oficios)):
    if type(df_oficios['Recebimento'][k]) == str or type(df_oficios['Recebimento'][k]) == float:
        idx2drop_list2.append(k)
    elif type(df_oficios['Recebimento'][k]) == int:
        idx2drop_list2.append(k)
        
print(idx2drop_list2)
df_oficios = df_oficios.drop(idx2drop_list2, axis = 0).reset_index(drop = True)

[]


In [25]:
df_oficios

,Recebimento,Assunto,Ofício,Advogado,Info,Status,line
0,2018-12-04,FWD: [6-5840000024315] Quebra de Sigilo - URGE...,NaN,Luisa,Caso novo,Enviado #4512,4512
1,2018-12-04,RE_ _1-9575000024540_ BR Data Request 00009613...,NaN,Rafael,Extensão #4193,Enviado advice #4515,4515
2,2018-12-04,FWD: [6-4781000024836] Afastamento do sigilo t...,NaN,Luisa,Caso novo,Enviado #4513,4513
3,2018-12-04,RES: [3-2530000024386] BR Data Request 0004435...,NaN,Rafael,Extensão #4358,Enviado advice #4516,4516
4,2018-12-04,Fwd: [juridicobrasil] Quebra de sigilo telemát...,NaN,Luisa,Email indicando autoridade #4481,enviado #4481,4481
...,...,...,...,...,...,...,...
5898,2019-12-27,FWD: [7-2066000028626] [BRA] - [Official Lette...,006-2019,Helena,relacionado #7739,respondido ao LIS,
5899,2019-12-26,Re: [6-6324000028920] - c 2019/12/27 - Decembe...,NaN,Helena,relacionado #7771,respondido ao LIS,
5900,2019-12-26,Re: [3-0569000028501] PETITION - c 2020/01/06 ...,NaN,Helena,relacionado #7774,respondido ao LIS,
5901,2019-12-27,FWD: [9-9053000029127] BR Data Request: 0245.1...,627-2019\nOficio Judicial - 0245.19.007487-3,Helena,caso novo,#7784,7784


In [26]:
df_oficios['Advogado'] = df_oficios['Advogado'].str.strip()

In [27]:
df_oficios['vinculo'] = df_oficios['Info'].apply(FindNumbers)

In [28]:
df_oficios

,Recebimento,Assunto,Ofício,Advogado,Info,Status,line,vinculo
0,2018-12-04,FWD: [6-5840000024315] Quebra de Sigilo - URGE...,NaN,Luisa,Caso novo,Enviado #4512,4512,[]
1,2018-12-04,RE_ _1-9575000024540_ BR Data Request 00009613...,NaN,Rafael,Extensão #4193,Enviado advice #4515,4515,[4193]
2,2018-12-04,FWD: [6-4781000024836] Afastamento do sigilo t...,NaN,Luisa,Caso novo,Enviado #4513,4513,[]
3,2018-12-04,RES: [3-2530000024386] BR Data Request 0004435...,NaN,Rafael,Extensão #4358,Enviado advice #4516,4516,[4358]
4,2018-12-04,Fwd: [juridicobrasil] Quebra de sigilo telemát...,NaN,Luisa,Email indicando autoridade #4481,enviado #4481,4481,[4481]
...,...,...,...,...,...,...,...,...
5898,2019-12-27,FWD: [7-2066000028626] [BRA] - [Official Lette...,006-2019,Helena,relacionado #7739,respondido ao LIS,,[7739]
5899,2019-12-26,Re: [6-6324000028920] - c 2019/12/27 - Decembe...,NaN,Helena,relacionado #7771,respondido ao LIS,,[7771]
5900,2019-12-26,Re: [3-0569000028501] PETITION - c 2020/01/06 ...,NaN,Helena,relacionado #7774,respondido ao LIS,,[7774]
5901,2019-12-27,FWD: [9-9053000029127] BR Data Request: 0245.1...,627-2019\nOficio Judicial - 0245.19.007487-3,Helena,caso novo,#7784,7784,[]


In [29]:
## saving the clean dataframe:
df_oficios.to_csv(os.path.join(path_output, 'oficios_google_clean.csv'), sep = '\t',\
               encoding = 'utf-8', index = False)

### Trix

In [30]:
## loading trix data (content/wiretap cases tab):
df_trix = pd.read_excel(os.path.join(path_data,'main_trix_v2.xlsx'), sheet_name = 0, skiprows = [0])


In [31]:
## dropping some empty lines:
df_trix.dropna(subset = ['Case # (DLBCA and Google reference)'])

,Unnamed: 0,Case # (DLBCA and Google reference),UID,Date of the Court Orders,Unnamed: 4,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,1,\n,NaN,NaN,\n,2018-01-17 00:00:00,Yes,0009460-19.2013.403.6181,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6-9080000028545,valdeci.silvacampos@gmail.com\nevangelina.pinh...,1st Court Order – 11.23.2012\n2nd Court Order ...,f,5th Federal Criminal Court of São Paulo,Yes,0002618-91.2011.403.6181,No,Google received a court order to provide conte...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,9-5821000001559 FPO Class Action Case,reichswehr88.blogspot.com.br,1st Court Order – 08.07.2013\n2nd Court Order ...,NaN,9th Federal Criminal Court of São Paulo,Yes,0007067-24.2013.403.6181,No,Google received court order to provide data/co...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5-6191000002205,alvarado.alejandro@gmail.com,Court Order - 11.07.2013,NaN,8th Criminal Court of Goiânia - Goiás State,No,201003908777,No,Google received court order to provide and pre...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,[9-9709000000828] ),quarne.ba@gmail.com\njosemarcos.quarne@gmail.com,1st Court Order – 03.22.2013\n2nd Court Order ...,NaN,2nd Criminal Court of Salvador - Bahia State,No,0076690-72.2011.805.0001,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10539,10542,[6-2387000030981],"alexxx -- Gmail, denxxx -- Gmail, helxxx -- Gm...",Court Order 2020/08/25\nReceived 2020/08/25,NaN,Vara Única da Comarca de Ferreira Gomes - AP,NaN,0000880-18.2020.8.03.0006,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10543,10546,[8-1172000030681],"ioaxxx -- Hotmail, romxxx -- Gmail, fraxxx -- ...",Court Order 2020/08/21\nReceived 2020/08/25,NaN,1ª Vara Criminal Federal de Vitória/ES,NaN,5005563-26.2020.4.02.5001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545,10548,[4-9100000031065],"22 IMEIs, 20 phone numbers",Court Order 2020/08/12 Received 2020/08/17,NaN,3ª Vara de Tóxicos da Comarca de Belo Horizont...,NaN,002419.104.234-0 (CNJ n. 1042340-66.2019.8.13....,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10546,10549,[3-5849000026727],"7 phone numbers, 5 IMEIS",Court Order 2019/05/07 Received 2020/08/26,NaN,1ª Vara de Delitos de Organizações Criminosas ...,NaN,014055-55.2019.8.06.0001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
## renaming some columns:
df_trix.rename(columns={'Unnamed: 0':'line'}, inplace=True)

In [33]:
df_trix

,line,Case # (DLBCA and Google reference),UID,Date of the Court Orders,Unnamed: 4,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,1,\n,NaN,NaN,\n,2018-01-17 00:00:00,Yes,0009460-19.2013.403.6181,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6-9080000028545,valdeci.silvacampos@gmail.com\nevangelina.pinh...,1st Court Order – 11.23.2012\n2nd Court Order ...,f,5th Federal Criminal Court of São Paulo,Yes,0002618-91.2011.403.6181,No,Google received a court order to provide conte...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,9-5821000001559 FPO Class Action Case,reichswehr88.blogspot.com.br,1st Court Order – 08.07.2013\n2nd Court Order ...,NaN,9th Federal Criminal Court of São Paulo,Yes,0007067-24.2013.403.6181,No,Google received court order to provide data/co...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5-6191000002205,alvarado.alejandro@gmail.com,Court Order - 11.07.2013,NaN,8th Criminal Court of Goiânia - Goiás State,No,201003908777,No,Google received court order to provide and pre...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,[9-9709000000828] ),quarne.ba@gmail.com\njosemarcos.quarne@gmail.com,1st Court Order – 03.22.2013\n2nd Court Order ...,NaN,2nd Criminal Court of Salvador - Bahia State,No,0076690-72.2011.805.0001,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10544,10547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545,10548,[4-9100000031065],"22 IMEIs, 20 phone numbers",Court Order 2020/08/12 Received 2020/08/17,NaN,3ª Vara de Tóxicos da Comarca de Belo Horizont...,NaN,002419.104.234-0 (CNJ n. 1042340-66.2019.8.13....,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10546,10549,[3-5849000026727],"7 phone numbers, 5 IMEIS",Court Order 2019/05/07 Received 2020/08/26,NaN,1ª Vara de Delitos de Organizações Criminosas ...,NaN,014055-55.2019.8.06.0001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10547,10550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
## avoiding some empty columns with no name:
df_trix = df_trix.loc[:, [column for column in df_trix.columns if 'Unnamed' not in column]]

In [35]:
## dropping all rows whose all columns are nan values:
df_trix  = df_trix.dropna(how = 'all')

In [36]:
df_trix

,line,Case # (DLBCA and Google reference),UID,Date of the Court Orders,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],STQ confirm scope of production,...,Previous Orders and Answers,Google's proposed answer,LIS: \nInformatin about case,LIS: \nPolicy and content check,LIS: \nGoogle Inc answer sent to authority? which one?,LIS:\nPriority,LIS:\nAssignee,Status,Status - writ of mandamus,Law Firm
0,1,\n,NaN,NaN,2018-01-17 00:00:00,Yes,0009460-19.2013.403.6181,No,NaN,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,"Andre reviewed and signed, but missing affidav...",Account status: live.,Speech: No CRC (Gmail) - No speech related inv...,May 23rd: Felipe shared content and letters wi...,P0,Completed\n(Felipe),LIS Status: May 22: waiting on Matt/Trev's dec...,A writ was not filed in this case,A writ was not filed in this case
1,2,6-9080000028545,valdeci.silvacampos@gmail.com\nevangelina.pinh...,1st Court Order – 11.23.2012\n2nd Court Order ...,5th Federal Criminal Court of São Paulo,Yes,0002618-91.2011.403.6181,No,Google received a court order to provide conte...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,NaN,Speech: Not enough information. Nature of inve...,*all content preserved is linked to matters re...,Content and letters shared with OC. Download o...,P0 - 04,Completed \n(Keyla 5/20),"50,000 BRL Daily fine and threat of contempt. ...",DLBCA filed a writ of mandamus before TRF-3. I...,DLBCA
2,3,9-5821000001559 FPO Class Action Case,reichswehr88.blogspot.com.br,1st Court Order – 08.07.2013\n2nd Court Order ...,9th Federal Criminal Court of São Paulo,Yes,0007067-24.2013.403.6181,No,Google received court order to provide data/co...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,https://docs.google.com/a/google.com/document/...,LIS preserved content.,Content: preserved for the same matter / Check...,Content and letters shared with OC. Download o...,P0 - 05,Completed \n(Felipe 5/19),"50,000 BRL daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TRF-3. D...,DLBCA
3,4,5-6191000002205,alvarado.alejandro@gmail.com,Court Order - 11.07.2013,8th Criminal Court of Goiânia - Goiás State,No,201003908777,No,Google received court order to provide and pre...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Content: CO asks for historical content + 15 d...,Content and letters shared with OC. Download o...,P0 - 03,Completed\n(Felipe 5/20),"10,000 BRL Daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TJGO. In...,DLBCA
4,5,[9-9709000000828] ),quarne.ba@gmail.com\njosemarcos.quarne@gmail.com,1st Court Order – 03.22.2013\n2nd Court Order ...,2nd Criminal Court of Salvador - Bahia State,No,0076690-72.2011.805.0001,No,NaN,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Speech: Investigation of crimes against the ta...,May 21: Content and letters shared with OC. Do...,P1 - 04,Completed \n(Felipe),"10,000 BRL daily fine and no threat of contemp...",Not filed writ in this case,Not filed writ in this case
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10544,10547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545,10548,[4-9100000031065],"22 IMEIs, 20 phone numbers",Court Order 2020/08/12 Received 2020/08/17,3ª Vara de Tóxicos da Comarca de Belo Horizont...,NaN,002419.104.234-0 (CNJ n. 1042340-66.2019.8.13....,NaN,NaN,LEGAL OPINION: \n \n1/ Background: N/A\n \n2/ ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10546,10549,[3-5849000026727],"7 phone numbers, 5 IMEIS",Court Order 2019/05/07 Received 2020/08/26,1ª Vara de Delitos de Organizações Criminosas ...,NaN,014055-55.2019.8.06.0001,NaN,NaN,LEGAL OPINION: \n \n1/ Background: N/A\n \n2/ ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10547,10550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...

In [37]:
df_trix_clean = df_trix.loc[:, df_trix.columns[1:]].dropna(how = 'all')

In [38]:
df_trix_clean

,Case # (DLBCA and Google reference),UID,Date of the Court Orders,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],STQ confirm scope of production,Deadline,...,Previous Orders and Answers,Google's proposed answer,LIS: \nInformatin about case,LIS: \nPolicy and content check,LIS: \nGoogle Inc answer sent to authority? which one?,LIS:\nPriority,LIS:\nAssignee,Status,Status - writ of mandamus,Law Firm
0,\n,NaN,NaN,2018-01-17 00:00:00,Yes,0009460-19.2013.403.6181,No,NaN,NaN,2014-02-28 00:00:00,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,"Andre reviewed and signed, but missing affidav...",Account status: live.,Speech: No CRC (Gmail) - No speech related inv...,May 23rd: Felipe shared content and letters wi...,P0,Completed\n(Felipe),LIS Status: May 22: waiting on Matt/Trev's dec...,A writ was not filed in this case,A writ was not filed in this case
1,6-9080000028545,valdeci.silvacampos@gmail.com\nevangelina.pinh...,1st Court Order – 11.23.2012\n2nd Court Order ...,5th Federal Criminal Court of São Paulo,Yes,0002618-91.2011.403.6181,No,Google received a court order to provide conte...,NaN,2013-11-28 00:00:00,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,NaN,Speech: Not enough information. Nature of inve...,*all content preserved is linked to matters re...,Content and letters shared with OC. Download o...,P0 - 04,Completed \n(Keyla 5/20),"50,000 BRL Daily fine and threat of contempt. ...",DLBCA filed a writ of mandamus before TRF-3. I...,DLBCA
2,9-5821000001559 FPO Class Action Case,reichswehr88.blogspot.com.br,1st Court Order – 08.07.2013\n2nd Court Order ...,9th Federal Criminal Court of São Paulo,Yes,0007067-24.2013.403.6181,No,Google received court order to provide data/co...,NaN,2013-10-22 00:00:00,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,https://docs.google.com/a/google.com/document/...,LIS preserved content.,Content: preserved for the same matter / Check...,Content and letters shared with OC. Download o...,P0 - 05,Completed \n(Felipe 5/19),"50,000 BRL daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TRF-3. D...,DLBCA
3,5-6191000002205,alvarado.alejandro@gmail.com,Court Order - 11.07.2013,8th Criminal Court of Goiânia - Goiás State,No,201003908777,No,Google received court order to provide and pre...,NaN,2013-11-22 00:00:00,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Content: CO asks for historical content + 15 d...,Content and letters shared with OC. Download o...,P0 - 03,Completed\n(Felipe 5/20),"10,000 BRL Daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TJGO. In...,DLBCA
4,[9-9709000000828] ),quarne.ba@gmail.com\njosemarcos.quarne@gmail.com,1st Court Order – 03.22.2013\n2nd Court Order ...,2nd Criminal Court of Salvador - Bahia State,No,0076690-72.2011.805.0001,No,NaN,NaN,2013-10-21 00:00:00,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Speech: Investigation of crimes against the ta...,May 21: Content and letters shared with OC. Do...,P1 - 04,Completed \n(Felipe),"10,000 BRL daily fine and no threat of contemp...",Not filed writ in this case,Not filed writ in this case
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10539,[6-2387000030981],"alexxx -- Gmail, denxxx -- Gmail, helxxx -- Gm...",Court Order 2020/08/25\nReceived 2020/08/25,Vara Única da Comarca de Ferreira Gomes - AP,NaN,0000880-18.2020.8.03.0006,NaN,NaN,LEGAL OPINION:\n \n1/ Background: n/a\n \n2/ L...,2020-09-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10543,[8-1172000030681],"ioaxxx -- Hotmail, romxxx -- Gmail, fraxxx -- ...",Court Order 2020/08/21\nReceived 2020/08/25,1ª Vara Criminal Federal de Vitória/ES,NaN,5005563-26.2020.4.02.5001,NaN,NaN,LEGAL OPINION: \n1/ Background: n/a\n \n2/ Leg...,2020-08-31 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545,[4-9

In [39]:
df_trix.columns

Index(['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm'],
      dtype='object')

In [40]:
df_trix = df_trix_clean.merge(df_trix, how = 'left' )

In [41]:
df_trix.columns

Index(['Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm', 'line'],
      dtype='object')

In [42]:
df_trix = df_trix.loc[:, ['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       "DLBCA's suggested answer", 'Previous Orders and Answers',
       "Google's proposed answer", 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm']]

In [43]:
### "Date of the Court Orders" field: break into multiple fields?
for k in random.sample(range(0, len(df_trix)-1), 10):
    print(df_trix['Date of the Court Orders'][k])
    print('----------')

Court Order
2020/06/02
Received
2020/06/03
----------
Court Order
2020/01/28
Received
2020/02/05
----------
Court Order
2020/06/04
Received
2020/06/15
----------
Court Order
2020/02/04
Received
2020/02/05
----------
Court Order
2019/07/05 received 2019/10/03
----------
Court Order
2020/05/07
Received
2020/05/08
----------
Court Order 2019/11/18 

 Received: 2019/11/19

----------
Court Order
2019/03/01 received 2019/03/01
----------
Court Order
2020/05/11
Received
2020/05/12
----------
Court Order
 2020/04/08
Received
2020/04/09
----------


In [44]:
### "Date of the Court Orders" field: break into multiple fields?
for k in random.sample(range(0, len(df_trix)-1), 10):
    print(df_trix['Deadline'][k])
    print('----------')

19.02.2018
----------
2020-08-03 00:00:00
----------
2019-03-07 00:00:00
----------
2016-03-14 00:00:00
----------
2019-07-09 00:00:00
----------
2019-08-05 00:00:00
----------
03.23.2015
----------
2020-07-23 00:00:00
----------
2016-10-13 00:00:00
----------
2019-12-09 00:00:00
----------


In [45]:
idxDel_list = []
for k in range(0, len(df_trix)):
    date = df_trix['Deadline'][k]
    
    if type(date) == float or type(date) == int or type(date) == str:
        if type(date) != str and np.isnan(date) == False:
            print(k, ' ', date)
            idxDel_list.append(k)
        elif type(date) == str:
            print(k, ' ', date)
            idxDel_list.append(k)            

12   No deadline
14   No deadline
19   No deadline
24   No deadline
26   Received 04/07/2014
35   No deadline
36   No Deadline
38   No Deadline 
41   No Deadline
46   No Deadline
47   No Deadline
54   No deadline
55   No Deadline
56   No Deadline
58   No deadline
60   No deadline
64   No deadline
65   No deadline
66   06.27.2014
67   07.25.2014
68   07.09.2014
69   06.24.2014
70   07.07.2014
71   07.15.2014
72   07.11.2014
73   07.01.2014 (24 hours and 2 hours for SMP)
74   1st Court Order 07.10.2014 2nd Court Order 07.25.2014
75   07.10.2014
76   07.04.2014 
77   07.10.2014
78   07.04.2014
80   07.10.2014
81   07.19.2014
82   07.17.2014
83   07.18.2014
84   07.19.2014
85   07.21.2014 (10 days)
86   07.30.2014
87   07.25.2014
88   07.28.2014
89   07.24.2014
90   08.12.2014
91   08.26.2014
92   08.14.2014
93   08.14.2014
94   08.15.2014
95   08.14.2014
96   08.19.2014
97   08.18.2014
98   08.24.2014
99   08.15.2014
100   08.22.2014
101   08.20.2014
102   08.26.2014
103   08.07.2014
104 

3934   2018.09.04
3935   2018.09.13
3936   2018.08.30
3937   2018.09.01
3938   2018.09.04
3939   2018.09.07
3940   2018.09.01
3941   2018.08.31
3942   2018.10.01
3943   2018.09.05
3944   2018.09.15
3945   2018.09.18
3946   2018.09.13
3947   2018.09.05
3948   2018.09.05
3949   2018.09.10
3950   2018.09.05
3951   2018.09.10
3952   2018.09.17
3953   2018.09.13
3954   2018.09.10
3955   2018.09.08
3956   2018.09.14
3957   2018.09.08
3958   2018.09.18
3959   2018.09.20
3960   2018.09.04
3961   2018.09.10
3962   2018.09.14
3963   2018.09.10
3964   2018.09.20
3965   2018.09.15
3966   2018.09.17
3967   2018.09.21
3968   2018.09.14
3969   2018.09.14
3970   2018.09.10
3971   2018.09.21
3972   2018.09.10
3973   2018.09.14
3974   2018.09.09
3975   2018.09.09
3976   2018.09.21
3977   2018.09.20
3978   2018.09.20
3979   2018.10.09
3980   2018.09.06
3981   2018.09.14
3982   2018.09.10
3983   2018.09.14
3984   2018.09.08
3985   2018.09.07
3986   2018.09.27
3987   2018.09.07
3988   2018.10.10
3989   201

4593   2019.01.02
4594   2018.12.30
4595   2019.01.07
4596   2019.01.07
4597   2018.12.30
4604   2019.01.06
4605   2019.01.14
4606   2019.01.09
4607   2018.01.23
4608   2019.01.14
4609   2019.01.14
4610   2018.12.13
4611   2019.01.14
4612   2019.01.14
4613   2019.01.10
4614   2019.01.15
4615   2019.01.15
4616   2019.01.15
4617   2019.01.15
4618   2019.01.29
4619   2019.01.18
4620   2019.01.15
4621   2019.01.13
4622   2019.01.10
4623   2019.01.28
4624   2019.01.13
4625   2019.01.13
4626   2019.01.23
4627   2019.01.01
4628   2019.01.18
4629   2019.01.17
4630   2019.01.21
4631   2019.01.24
4632   2018.01.16
4633   2019.01.24
4634   2018.12.19
4635   2019.01.25
4636   2019.01.16
4637   2019.02.01
4638   2019.01.21
4639   2019.01.21
4640   2019.01.21
4641   2019.01.19
4642   2019.02.04
4643   2019.01.28
4644   2019.01.22
4645   2019.01.22
4646   2019.01.28
4647   2019.01.22
4648   2019.01.19
4649   2019.01.19
4650   2019.01.22
4651   2019.01.22
4652   2019.01.08
4653   2019.01.25
4654   201

In [46]:
df_trix.shape

(10310, 22)

In [47]:
df_trix['Deadline'].value_counts()

2020-07-20 00:00:00    50
2020-07-27 00:00:00    48
2020-06-08 00:00:00    46
2020-07-06 00:00:00    44
2020-06-22 00:00:00    43
                       ..
2020-02-15 00:00:00     1
04.02.2018              1
2016-04-07 00:00:00     1
2018-01-23 00:00:00     1
0                       1
Name: Deadline, Length: 1935, dtype: int64

In [48]:
for idx in idxDel_list:
    print(idx)
    df_trix.at[idx, 'Deadline'] = np.nan
    

12
14
19
24
26
35
36
38
41
46
47
54
55
56
58
60
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
108
109
110
113
114
115
116
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
206
207
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
244
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
287
288
295
297
298
299
300
301
302
303
305
306
308
309
310
810
865
866
868
1160
1231
1337
1353
1467
1758
1770
1879
1919
1920
1934
2037
2044
2091
2455


In [49]:
#df_trix['Deadline'] = pd.to_datetime(df_trix['Deadline'])

In [50]:
df_trix

,line,Case # (DLBCA and Google reference),UID,Date of the Court Orders,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],STQ confirm scope of production,...,Previous Orders and Answers,Google's proposed answer,LIS: \nInformatin about case,LIS: \nPolicy and content check,LIS: \nGoogle Inc answer sent to authority? which one?,LIS:\nPriority,LIS:\nAssignee,Status,Status - writ of mandamus,Law Firm
0,1,\n,NaN,NaN,2018-01-17 00:00:00,Yes,0009460-19.2013.403.6181,No,NaN,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,"Andre reviewed and signed, but missing affidav...",Account status: live.,Speech: No CRC (Gmail) - No speech related inv...,May 23rd: Felipe shared content and letters wi...,P0,Completed\n(Felipe),LIS Status: May 22: waiting on Matt/Trev's dec...,A writ was not filed in this case,A writ was not filed in this case
1,2,6-9080000028545,valdeci.silvacampos@gmail.com\nevangelina.pinh...,1st Court Order – 11.23.2012\n2nd Court Order ...,5th Federal Criminal Court of São Paulo,Yes,0002618-91.2011.403.6181,No,Google received a court order to provide conte...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,NaN,Speech: Not enough information. Nature of inve...,*all content preserved is linked to matters re...,Content and letters shared with OC. Download o...,P0 - 04,Completed \n(Keyla 5/20),"50,000 BRL Daily fine and threat of contempt. ...",DLBCA filed a writ of mandamus before TRF-3. I...,DLBCA
2,3,9-5821000001559 FPO Class Action Case,reichswehr88.blogspot.com.br,1st Court Order – 08.07.2013\n2nd Court Order ...,9th Federal Criminal Court of São Paulo,Yes,0007067-24.2013.403.6181,No,Google received court order to provide data/co...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,https://docs.google.com/a/google.com/document/...,LIS preserved content.,Content: preserved for the same matter / Check...,Content and letters shared with OC. Download o...,P0 - 05,Completed \n(Felipe 5/19),"50,000 BRL daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TRF-3. D...,DLBCA
3,4,5-6191000002205,alvarado.alejandro@gmail.com,Court Order - 11.07.2013,8th Criminal Court of Goiânia - Goiás State,No,201003908777,No,Google received court order to provide and pre...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Content: CO asks for historical content + 15 d...,Content and letters shared with OC. Download o...,P0 - 03,Completed\n(Felipe 5/20),"10,000 BRL Daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TJGO. In...,DLBCA
4,5,[9-9709000000828] ),quarne.ba@gmail.com\njosemarcos.quarne@gmail.com,1st Court Order – 03.22.2013\n2nd Court Order ...,2nd Criminal Court of Salvador - Bahia State,No,0076690-72.2011.805.0001,No,NaN,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Speech: Investigation of crimes against the ta...,May 21: Content and letters shared with OC. Do...,P1 - 04,Completed \n(Felipe),"10,000 BRL daily fine and no threat of contemp...",Not filed writ in this case,Not filed writ in this case
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10305,10542,[6-2387000030981],"alexxx -- Gmail, denxxx -- Gmail, helxxx -- Gm...",Court Order 2020/08/25\nReceived 2020/08/25,Vara Única da Comarca de Ferreira Gomes - AP,NaN,0000880-18.2020.8.03.0006,NaN,NaN,LEGAL OPINION:\n \n1/ Background: n/a\n \n2/ L...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10306,10546,[8-1172000030681],"ioaxxx -- Hotmail, romxxx -- Gmail, fraxxx -- ...",Court Order 2020/08/21\nReceived 2020/08/25,1ª Vara Criminal Federal de Vitória/ES,NaN,5005563-26.2020.4.02.5001,NaN,NaN,LEGAL OPINION: \n1/ Background: n/a\n \n2/ Leg...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10307,10548,[4-9100000031065],"22 IMEIs, 20 phone numbers",Court Order 2020/08/12 Received 2020/08/17,3ª Vara de Tóxicos da Comarca 

In [51]:
## saving the clean dataframe:
df_trix.to_csv(os.path.join(path_output, 'main_trix_clean_v2.csv'), sep = '\t',\
               encoding = 'utf-8', index = False)

In [55]:
## loading trix data (BoletosAds):
df_trix2 = pd.read_excel(os.path.join(path_data,'main_trix.xlsx'), sheet_name = 1)


In [56]:
df_trix2

,Case # (CQ reference)/thread LIS,Medium/low priority,Court/LE,Lawsuit number,Boleto number/target,Date received,Date received CQ,Response is good to go,Deadeline,Response delivered,Registration data provided (yes/no),Follow up
0,1 - [6-8478000025734],low,Delegacia de Polícia Civil de Bastos/SP,BO nº 205/2019,34191.98068.34844.500008.02031.195403.1.782200...,2019-04-01,2019-03-28,yes,not informed - internal deadline 2019/04/11,Petition filed on 2019/04/23,yes,"2019/07/05\nInquérito arquivado, infos suficie..."
1,2 - [7-2117000025786],low,1º Distrito Policial de Marília/SP,Inquérito Policial nº 343/18,34191.98035.63856.800000.60203.11959.089737.1....,2019-04-01,2019-04-01,yes,not informed - internal deadline 2019/04/11,Petition filed on 2019/05/21,yes,"2019/07/05\nInquérito em andamento, infos rece..."
2,3 - [6-8377000025563],low,8ª Subdivisão Policial de Paranavaí/PR,Inquérito Policial nº 105862/2016,34193695100012000001980151093000000203119580,2019-03-20,2019-03-20,no,not informed - internal deadline 2019/04/11,petition filed on 2019/04/30,not yet,monthly
3,4 -,medium: subpoena for Google's Brazil represent...,78º Distrito Policial de São Paulo/SP,Inquérito Policial nº 604/2016,34191.98001.58863.90005.02031.195403.5.6721000...,2019-03-29,2019-03-29,no,2019-04-15 00:00:00,Petition filed on 2019/06/25,Yes,monthly
4,5 - [6-7163000025634],low,154 Distrito Policial do RJ - Cordeiro,Registro de Ocorrência nº 154-00771/2018,34191.98050.3046810000 02031195809 2 76590000 ...,2019-04-24,2019-04-24,yes,2019-05-06 00:00:00,Petition filed on 2019/05/30,Yes,"2019/09/20\nInquérito em andamento, infos sufi..."
5,6 - [3-4773000026501],low,Delegacia de Polícia Civil de Leme/SP,Registro de Ocorrência nº 254/19,3419198068 65176800004 02031195007 2 788400000...,2019-05-13,2019-05-14,yes,2019-05-20 00:00:00,Petition filed on 2019/07/12,Yes,"2019/09/20\nInquérito em andamento, infos não ..."
6,7 - [3-6688000026511],low,18º Distrito Policial do Alto da Mooca - SP,BO nº 2589/2019,34191.98068.47224.100009.02031.195502.l.784100...,2019-05-15,2019-05-16,yes,2019-05-20 00:00:00,Petition filed on 2019/07/05,Yes,-
7,8 - [5-2791000026705],low,Delegacia de Polícia de Buritama/SP,BO nº 332/19,34191.98068 35272.000007 02031.195304 3 782300...,2019-06-03,2019-06-06,yes,2019-06-11 00:00:00,Petition filed on 2019/07/11,Yes,"2019/09/17\nAinda não há Inquérito, infos sufi..."
8,9 -,low,14ª Delegacia de Polícia de Pinheiros/SP - DIPO 4,IP 890/2018\nAutos n. 0109050-78.2018.8.26.0050,34191 .98050 33164.500002 02031.195007 2 76650...,2019-07-03,2019-07-03,no,2019-07-08 00:00:00,Petition filed on 2019/08/01,yes,2019/11/29\nIP em cartório com o escrivão Robi...
9,10 - [0-8549000026934],low,Núcleo de Análise de Inquéritos Policiais de C...,Inquêrito Policial nº 0004585-16.2015.8.16.0013,34191980018558750000101031195207259880000675345,2019-07-09,2019-07-15,yes,2019-07-15 00:00:00,-,Yes,"2019/12/03\nIP em andamento, infos suficientes"


In [61]:
## saving the clean dataframe:
df_trix2.to_csv(os.path.join(path_output, 'MainTrix_BoletosAds.csv'), sep = '\t',\
               encoding = 'utf-8', index = False)

In [57]:
## loading trix data (Writs of mandamus "detenção"):
df_trix3 = pd.read_excel(os.path.join(path_data,'main_trix.xlsx'), sheet_name = 2)

In [58]:
df_trix3

,Case #,UID,Date of the Court Orders,Kwyjibo Reference #,Court,Lawsuit Number,Legal issue/Summary of the request/Data produced,Writ filed by CQ/date,Daily fines,Freezing of assets/date,deadline to file a writ against the freezing of assets decision,Status
0,#2269,espanholufac -- Gmail,2017-07-12 00:00:00,7-5706000020005/3-1476000018410,Tribunal Regional Federal da 1ª Região,1008782-18.2017.4.01.0000,CO determined Google to provide access logs an...,2017-10-23 00:00:00,no,no,NaN,The writ was denied on 2019/11/27. We are prep...
1,#2382,z3xxx -- Gmail; eixxx -- Gmail; https://plus.g...,2017-07-27 00:00:00,9-8871000018612,Tribunal Regional Federal da 1ª Região,1017556-03.2018.4.01.0000,CO determined Google to provide registration d...,2018-06-25 00:00:00,"R$ 100.000,00","*Court determined Google to pay R$ 29.000.000,...",Although no freezing of assest has occured so ...,The writ was denied on 2019/05/16. We filed an...
2,#4370,isxxx -- Gmail,2018-11-01 00:00:00,3-2412000023860,Tribunal de Justiça do Estado de São Paulo,2069123-90.2019.8.26.0000,CO determined Google to provide registration d...,2019/28/03,"R$ 20.000,00",no,NaN,The writ was denied on 2019/11/18. The CO was ...
3,#4942,enxxx -- Gmail,2019-02-04 00:00:00,9-3434000021949,Tribunal de Justiça do Estado de São Paulo,2141947-47.2019.8.26.0000,CO determined Google to provide registration d...,2019-06-27 00:00:00,no,no,NaN,The writ was denied on 2019/08/30. We filed an...
4,#5412,nerxxx -- Gmail,2019-04-29 00:00:00,4-7597000026554,Tribunal de Justiça do Estado de São Paulo,2150024-45.2019.8.26.0000,CO determined Google to provide registration d...,2019-07-10 00:00:00,R$ 200.000.00,"R$ 3,570,000.00",Writ already filed by BFBM to challenge the fine.,The writ was denied on 2019/11/18. We are pre...
5,#5507,dr.xxx -- Gmail,2019-03-27 00:00:00,0-1801000026267,Tribunal Regional Federal da 4ª Região,5032902-05.2019.4.04.0000,CO determined Google to provide Gmail content ...,2019-07-31 00:00:00,"R$ 60,000.00","R$ 1,860,000,00",2019-12-19 00:00:00,The writ was denied on 2019/10/15. The CO was ...
6,#5598,secxxx -- Gmail,2019/05/06 (DNE account)\n2019/08/05\n2019/11/...,0-8735000026542\n7-4585000028555,Tribunal de Justiça do Estado de São Paulo,\n2279290-85.2019.8.26.0000\n,CO determined Google to provide registration d...,2019-12-11 00:00:00,"R$ 10,000.00",NaN,NaN,We filed the writ with a preliminary injunctio...
7,#6333,rioxxx -- Gmail,2019-07-10 00:00:00,9-6991000027003/7-9061000029035/1-3174000028603,Tribunal de Justiça do Estado de Sergipe,201900138033,CO determined Google to provide registration d...,2019-11-26 00:00:00,no,no,NaN,The writ was filled on 2019/11/16. and has not...
8,#6449,soaxxx -- Gmail,2019-07-01 00:00:00,4-3140000027281/2-4871000027790,Tribunal de Justiça do Estado de São Paulo,22745005820198260000,CO determined Google to provide registration d...,2019-12-05 00:00:00,"R$ 10.000,00",no,NaN,The writ was filled on 2019/12/05 with a reque...


In [62]:
## saving the clean dataframe:
df_trix3.to_csv(os.path.join(path_output, 'MainTrix_Writs.csv'), sep = '\t',\
               encoding = 'utf-8', index = False)

In [59]:
## loading trix data (Writs of mandamus "detenção"):
df_trix4 = pd.read_excel(os.path.join(path_data,'main_trix.xlsx'), sheet_name = 3)

In [60]:
df_trix4

,Case #,Google reference,UID,Date of the Court Orders,Kwyjibo Reference #,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],...,Previous Orders and Answers,Google's proposed answer,LIS: \nInformatin about case,LIS: \nPolicy and content check,LIS: \nGoogle Inc answer sent to authority? which one?,LIS:\nPriority,LIS:\nAssignee,Status,Unnamed: 21,Unnamed: 22
0,44,1279397 / 6-8832000003411,constantine@gmail.com,2014-05-13 00:00:00,NaN,1st Judicial Court of Adamantina - São Paulo S...,No,51/13,Unknown,No IP resolves to Brazil for this account. It...,...,https://drive.google.com/file/d/0B24XiDkZhwlod...,NaN,same lawsuit of line 41 - different accounts....,#41 and #44 are related to the same investigat...,"June 11: produced on case #41, closing #44 unt...",NaN,NaN,Petition filed on 07.07.2014.,NaN,NaN
1,49,1281880 /(LS),rafael.seckersantana@gmail.com\narquivossenhas...,2014-05-19 00:00:00,NaN,5th Federal Criminal Court of São Paulo,YES\n\n(same court and same Judge),5th Federal Criminal Court of São Paulo,Unknown,Ordered to produce all IP/reg info; provide al...,...,https://drive.google.com/file/d/0B24XiDkZhwloS...,NaN,NaN,NaN,May 28th: LIS produced / shared content with O...,P0,NaN,Same judge that has the charges against Fabi a...,not filed writ in this case,not filed writ in this case
2,55,1210183\n1-0230000002988,sergioartstar@gmail.com\njosiania.mascarenhas0...,2/25/2014\n04/01/2014,NaN,14th Criminal Court of Curitiba,No,2013.34683-3,No,Ordered to produce future content only which i...,...,https://drive.google.com/a/dlbca.com.br/file/d...,NaN,Not eligible for CRC\nNo indication of Speech ...,sergioartstar - full nexus to BRA\njosianiamas...,Produce on the non FIP accounts. Production le...,NaN,Felipe,Production letter shared with OC on July 2. Pr...,Petition filed on 07.23.2014.,not filed writ in this case
3,76,1247695,gustavorosado@gmail.com produção@gondimgarcia....,1st Court Order - 06.17.2014 ...,459635.0,2nd Criminal Court of Mossoró - Rio Grande do ...,No,2013316,Unknown,"Same case of #65, ordered (i) a 15 days wireta...",...,https://drive.google.com/a/dlbca.com.br/?tab=m...,NaN,NaN,NaN,NaN,NaN,NaN,Petition filed on 07.30.2014.,Not filed writ in this case,Not filed writ in this case
4,84,1300371,jkillick01@googlemail.com\njkillick01@gmail.com,06.24.2014 (Date of the chief of police reques...,NaN,Chief of Police of Rio de Janeiro State,No,906-01786/2014,NaN,"Requested (i) Registration data, Logs and crea...",...,https://docs.google.com/a/dlbca.com.br/file/d/...,NaN,July 2: Thank you for your message. The activi...,NaN,NaN,NaN,NaN,LIS responded July 2,Not filed writ in this case,Not filed writ in this case
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1533,NaN,"plus.google.com/+Bxxx -- Google Plus, youtube....",Court order 06.13.2016 received on 09.20.2016.,NaN,2ª Vara Criminal de Bangu - Rio de Janeiro,NaN,0014972-81.2016.8.19.0204,NaN,Order to (i) inform the Google ID associated w...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response sent via e-mail.Petition filed on 10....,NaN,NaN
59,1563,NaN,"jefxxxx@gmail.com, bxxxx@gmail.com, clxxxx@gma...",Court order 09.19.2016 received on 09.28.2016.,NaN,1ª Vara Criminal de São João de Meriti - Rio d...,NaN,0030826-51.2014.8.19.0054,NaN,1) Stored content:\nOrder to provide the foll...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,1616,NaN,chxxxx@gmail.com,Court order 10.14.2016 received on 10.19.2016.,NaN,1ª Vara Federal Criminal de Vitória - Espírito...,NaN,0501797-32.2016.4.02.5001,NaN,Order to provide (i) future content for the ac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response sent via SFTP and via e-mail.,NaN,NaN
61,1626,NaN,"123xxx@gmail.com, azaxxx@gmail.com, verxxx@gma...",Court order 06.27.2016 received on 10.21.2016.,NaN,3ª Vara Federal de Rio Branco - Acre,NaN,3885-19.2016.4.01.3000,NaN,Order to provide stored content for the target...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Production shared via SFTP.,Na

In [63]:
## saving the clean dataframe:
df_trix4.to_csv(os.path.join(path_output, 'MainTrix_Segmentation.csv'), sep = '\t',\
               encoding = 'utf-8', index = False)

#### "Controle de linhas" spreadsheet

In [163]:
## loading "controle de linhas" data (sheet "Controle de Linhas ok"):
df_controle = pd.read_excel(os.path.join(path_data,'controle_linhas.xlsx'),\
                            sheet_name = 'Controle de Linhas ok')


In [164]:
df_controle = df_controle.dropna(how = 'all').reset_index(drop = True)


In [165]:
df_controle.rename(columns = {'Nº da linha':'line'}, inplace = True)

In [166]:
df_controle.rename(columns = {'\n':'Advogado'}, inplace = True)

In [167]:
df_controle

,Advogado,line,Extensão,Nº Principal do Processo,Objeto,Observações ref. ao objeto,Localização,UF,Prazo MS,Deadline,Data,Status/Estratégia,Solicitações e Considerações (Carina),Peticionar?,Data do Protocolo,Acompanhamento?
0,Helena,#6698,NaN,5042387-09.2019.4.02.5101,future data,NaN,8ª Vara Federal Criminal do Rio de Janeiro,RJ,2020-01-01 00:00:00,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Helena,#6700,NaN,0005384-85.2019.8.16.0153,future data,NaN,Vara Criminal de Santo Antônio da Platina - PR,PR,2020-01-01 00:00:00,2019-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,Mohamad,#6705,NaN,1000531-59.2019.4.04.4100,future data,NaN,3ª Vara Criminal e Execução Penal da Seção Jud...,RO,2020-01-01 00:00:00,2019-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Helena,#6714,NaN,0105844-13.2019.8.21.0001 (001/2.19.0068301-4),passwords,NaN,17ª Vara Criminal do Foro Central de Comarca d...,RS,2020-02-01 00:00:00,2019-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,Mohamad,#6713,NaN,0001588-12.2019.8.26.0411,future data,NaN,1ª Vara da Comarca de Pacaembu - SP,SP,2020-02-01 00:00:00,2019-09-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
5,Helena,#6715,NaN,0009706-23.2019.8.14.0051,future data,NaN,2ª Vara Criminal de Santarém/PA,PA,2020-02-01 00:00:00,2019-09-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
6,Mohamad,#6717,NaN,0012594-94.2019.8.26.0482,future data,NaN,3ª Vara Criminal da Comarca de Presidente Prud...,SP,2020-02-01 00:00:00,2019-09-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
7,Rafael,#6723,NaN,IPL 0518/2017-1 SR/PF/RJ,authority,NaN,DELEFAZ - Superintendência Regional no Rio de ...,RJ,27/12/2019,2019-09-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
8,Mohamad,#6731,NaN,5007126-43.2019.4.04.7003,future data,NaN,3ª Vara Federal de Maringá - PR,PR,2020-03-01 00:00:00,2019-09-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
9,Mohamad,#6735,NaN,5004129-84.2019.4.04.7004/PR,future data,NaN,1ª Vara Federal de Umuarama - PR,PR,2020-03-01 00:00:00,2019-09-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
df_controle['line'] = df_controle['line'].str.replace('#', '')

In [169]:
df_controle

,Advogado,line,Extensão,Nº Principal do Processo,Objeto,Observações ref. ao objeto,Localização,UF,Prazo MS,Deadline,Data,Status/Estratégia,Solicitações e Considerações (Carina),Peticionar?,Data do Protocolo,Acompanhamento?
0,Helena,6698,NaN,5042387-09.2019.4.02.5101,future data,NaN,8ª Vara Federal Criminal do Rio de Janeiro,RJ,2020-01-01 00:00:00,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Helena,6700,NaN,0005384-85.2019.8.16.0153,future data,NaN,Vara Criminal de Santo Antônio da Platina - PR,PR,2020-01-01 00:00:00,2019-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,Mohamad,6705,NaN,1000531-59.2019.4.04.4100,future data,NaN,3ª Vara Criminal e Execução Penal da Seção Jud...,RO,2020-01-01 00:00:00,2019-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Helena,6714,NaN,0105844-13.2019.8.21.0001 (001/2.19.0068301-4),passwords,NaN,17ª Vara Criminal do Foro Central de Comarca d...,RS,2020-02-01 00:00:00,2019-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,Mohamad,6713,NaN,0001588-12.2019.8.26.0411,future data,NaN,1ª Vara da Comarca de Pacaembu - SP,SP,2020-02-01 00:00:00,2019-09-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
5,Helena,6715,NaN,0009706-23.2019.8.14.0051,future data,NaN,2ª Vara Criminal de Santarém/PA,PA,2020-02-01 00:00:00,2019-09-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
6,Mohamad,6717,NaN,0012594-94.2019.8.26.0482,future data,NaN,3ª Vara Criminal da Comarca de Presidente Prud...,SP,2020-02-01 00:00:00,2019-09-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
7,Rafael,6723,NaN,IPL 0518/2017-1 SR/PF/RJ,authority,NaN,DELEFAZ - Superintendência Regional no Rio de ...,RJ,27/12/2019,2019-09-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
8,Mohamad,6731,NaN,5007126-43.2019.4.04.7003,future data,NaN,3ª Vara Federal de Maringá - PR,PR,2020-03-01 00:00:00,2019-09-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
9,Mohamad,6735,NaN,5004129-84.2019.4.04.7004/PR,future data,NaN,1ª Vara Federal de Umuarama - PR,PR,2020-03-01 00:00:00,2019-09-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
## looking for problemas in the Deadline column:
for k in range(0, len(df_controle)):
    date = df_controle['Deadline'][k]
    
    if type(date) == str or type(date) == float or type(date) == int:
        print(date)

nan
nan
22/10/2019
2019/11//20
nan
nan


In [171]:
df_controle

,Advogado,line,Extensão,Nº Principal do Processo,Objeto,Observações ref. ao objeto,Localização,UF,Prazo MS,Deadline,Data,Status/Estratégia,Solicitações e Considerações (Carina),Peticionar?,Data do Protocolo,Acompanhamento?
0,Helena,6698,NaN,5042387-09.2019.4.02.5101,future data,NaN,8ª Vara Federal Criminal do Rio de Janeiro,RJ,2020-01-01 00:00:00,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Helena,6700,NaN,0005384-85.2019.8.16.0153,future data,NaN,Vara Criminal de Santo Antônio da Platina - PR,PR,2020-01-01 00:00:00,2019-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,Mohamad,6705,NaN,1000531-59.2019.4.04.4100,future data,NaN,3ª Vara Criminal e Execução Penal da Seção Jud...,RO,2020-01-01 00:00:00,2019-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Helena,6714,NaN,0105844-13.2019.8.21.0001 (001/2.19.0068301-4),passwords,NaN,17ª Vara Criminal do Foro Central de Comarca d...,RS,2020-02-01 00:00:00,2019-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,Mohamad,6713,NaN,0001588-12.2019.8.26.0411,future data,NaN,1ª Vara da Comarca de Pacaembu - SP,SP,2020-02-01 00:00:00,2019-09-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
5,Helena,6715,NaN,0009706-23.2019.8.14.0051,future data,NaN,2ª Vara Criminal de Santarém/PA,PA,2020-02-01 00:00:00,2019-09-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
6,Mohamad,6717,NaN,0012594-94.2019.8.26.0482,future data,NaN,3ª Vara Criminal da Comarca de Presidente Prud...,SP,2020-02-01 00:00:00,2019-09-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
7,Rafael,6723,NaN,IPL 0518/2017-1 SR/PF/RJ,authority,NaN,DELEFAZ - Superintendência Regional no Rio de ...,RJ,27/12/2019,2019-09-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
8,Mohamad,6731,NaN,5007126-43.2019.4.04.7003,future data,NaN,3ª Vara Federal de Maringá - PR,PR,2020-03-01 00:00:00,2019-09-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
9,Mohamad,6735,NaN,5004129-84.2019.4.04.7004/PR,future data,NaN,1ª Vara Federal de Umuarama - PR,PR,2020-03-01 00:00:00,2019-09-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df_controle['Deadline'] = df_controle['Deadline'].str.replace('//', '/')

In [116]:
df_controle

,Advogado,line,Extensão,Nº Principal do Processo,Objeto,Observações ref. ao objeto,Localização,UF,Prazo MS,Deadline,Data,Status/Estratégia,Solicitações e Considerações (Carina),Peticionar?,Data do Protocolo,Acompanhamento?
0,Helena,6698,NaN,5042387-09.2019.4.02.5101,future data,NaN,8ª Vara Federal Criminal do Rio de Janeiro,RJ,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Helena,6700,NaN,0005384-85.2019.8.16.0153,future data,NaN,Vara Criminal de Santo Antônio da Platina - PR,PR,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mohamad,6705,NaN,1000531-59.2019.4.04.4100,future data,NaN,3ª Vara Criminal e Execução Penal da Seção Jud...,RO,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Helena,6714,NaN,0105844-13.2019.8.21.0001 (001/2.19.0068301-4),passwords,NaN,17ª Vara Criminal do Foro Central de Comarca d...,RS,2020-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mohamad,6713,NaN,0001588-12.2019.8.26.0411,future data,NaN,1ª Vara da Comarca de Pacaembu - SP,SP,2020-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Helena,6715,NaN,0009706-23.2019.8.14.0051,future data,NaN,2ª Vara Criminal de Santarém/PA,PA,2020-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Mohamad,6717,NaN,0012594-94.2019.8.26.0482,future data,NaN,3ª Vara Criminal da Comarca de Presidente Prud...,SP,2020-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Rafael,6723,NaN,IPL 0518/2017-1 SR/PF/RJ,authority,NaN,DELEFAZ - Superintendência Regional no Rio de ...,RJ,27/12/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Mohamad,6731,NaN,5007126-43.2019.4.04.7003,future data,NaN,3ª Vara Federal de Maringá - PR,PR,2020-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Mohamad,6735,NaN,5004129-84.2019.4.04.7004/PR,future data,NaN,1ª Vara Federal de Umuarama - PR,PR,2020-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
## converting to datetime:
df_controle['Prazo MS'] = pd.to_datetime(df_controle['Prazo MS'])
df_controle['Deadline'] = pd.to_datetime(df_controle['Deadline'])

In [58]:
df_controle

,Advogado,line,Extensão,Nº Principal do Processo,Objeto,Observações ref. ao objeto,Localização,UF,Prazo MS,Deadline,Data,Status/Estratégia,Solicitações e Considerações (Carina),Peticionar?,Data do Protocolo,Acompanhamento?
0,Helena,6698,NaN,5042387-09.2019.4.02.5101,future data,NaN,8ª Vara Federal Criminal do Rio de Janeiro,RJ,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Helena,6700,NaN,0005384-85.2019.8.16.0153,future data,NaN,Vara Criminal de Santo Antônio da Platina - PR,PR,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mohamad,6705,NaN,1000531-59.2019.4.04.4100,future data,NaN,3ª Vara Criminal e Execução Penal da Seção Jud...,RO,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Helena,6714,NaN,0105844-13.2019.8.21.0001 (001/2.19.0068301-4),passwords,NaN,17ª Vara Criminal do Foro Central de Comarca d...,RS,2020-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mohamad,6713,NaN,0001588-12.2019.8.26.0411,future data,NaN,1ª Vara da Comarca de Pacaembu - SP,SP,2020-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Helena,6715,NaN,0009706-23.2019.8.14.0051,future data,NaN,2ª Vara Criminal de Santarém/PA,PA,2020-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Mohamad,6717,NaN,0012594-94.2019.8.26.0482,future data,NaN,3ª Vara Criminal da Comarca de Presidente Prud...,SP,2020-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Rafael,6723,NaN,IPL 0518/2017-1 SR/PF/RJ,authority,NaN,DELEFAZ - Superintendência Regional no Rio de ...,RJ,2019-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Mohamad,6731,NaN,5007126-43.2019.4.04.7003,future data,NaN,3ª Vara Federal de Maringá - PR,PR,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Mohamad,6735,NaN,5004129-84.2019.4.04.7004/PR,future data,NaN,1ª Vara Federal de Umuarama - PR,PR,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
## saving the clean dataframe:
df_controle.to_csv(os.path.join(path_output, 'controle_linhas_clean.csv'), sep = '\t',\
                   encoding = 'utf-8', index = False)